# Description of analysis
In this notebook, we aim to look at engagement of posts, and whether posting at specific days of the week and timings of the day lead to greater engagement rate. To do so, we have to define the metric for "engagement".<br><br>
We face certain limitations and concerns when defining engagement of a post on a particular date. We address this points below and come up with our own engagement metric.

# Limitation/ concerns
This measurement of engagement comes with two limitations/threats to validity:<br>
<ol>
    <li>Likes of posts will rise will number of followrs. While the ideal metric would be: <i>Number of likes divided by number of followers at the time of posting</i>, instagram does not allow us to retrieve historical number of followers.</li>
    <li>If we look at absolute number of likes per day, e.g. all likes gathered on posts on Thursdays, we may have bias, where if Fukudon posts regularly on Thursdays, Thursdays may gain a higher engagement rate</li>
</ol>

# Mitigation
We first construct a linear regressor to fit likes across all dates, and look at the the percentage variance of the post to the regressor line. This is the engagement per post. We do this for all posts, and get the average engagement per post over a date.

# Formula for "Engagement"
In conclusion, we come up with the following formula:<h3><i>Engagement per day: (sum of all percentage variance for a day) divided by (number of posts on that day)</i></h3>


# Import relevant libraries and files

In [71]:
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime, timedelta, date
import calendar

In [72]:
instagram_df = pd.read_csv("../data/instagram_posts.csv")

In [73]:
instagram_df.head()

,Unnamed: 0,node.display_url,node.edge_media_to_caption.edges,node.owner.full_name,node.location.name,node.location.slug,hashtags,Likes,Date,Datetime
0,57,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,Let us bring you behind the scenes and show yo...,Fukudon | Donburi Specialist,Fukudon,fukudon,NaN,29,7/2/2022,7/2/2022 15:20
1,56,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,"🐯新年快乐 恭喜发财🍊\n\nFrom all of us at Fúkudon, we w...",Fukudon | Donburi Specialist,NaN,NaN,NaN,16,3/2/2022,3/2/2022 11:24
2,55,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,We’ve heard you! We are pleased to finally ann...,Fukudon | Donburi Specialist,Fukudon,fukudon,NaN,18,28/1/2022,28/1/2022 18:04
3,54,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,🐯🧧 Let us usher in a Roarsome Lunar New Year t...,Fukudon | Donburi Specialist,NaN,NaN,NaN,17,26/1/2022,26/1/2022 13:37
4,53,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,🧧Introducing to you our Roaring Combo this Lun...,Fukudon | Donburi Specialist,NaN,NaN,NaN,47,23/1/2022,23/1/2022 16:01


In [74]:
instagram_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Unnamed: 0                        58 non-null     int64 
 1   node.display_url                  55 non-null     object
 2   node.edge_media_to_caption.edges  58 non-null     object
 3   node.owner.full_name              55 non-null     object
 4   node.location.name                39 non-null     object
 5   node.location.slug                39 non-null     object
 6   hashtags                          9 non-null      object
 7   Likes                             58 non-null     int64 
 8   Date                              58 non-null     object
 9   Datetime                          58 non-null     object
dtypes: int64(2), object(8)
memory usage: 4.7+ KB


# Sort instagram posts by date

In [75]:
def get_date_from_string(date_string):
    return datetime.strptime(date_string, "%d/%m/%Y").date()

instagram_df["date_obj"] = instagram_df["Date"].apply(get_date_from_string)

In [76]:
instagram_df = instagram_df.sort_values(by="date_obj")
instagram_df.head()

,Unnamed: 0,node.display_url,node.edge_media_to_caption.edges,node.owner.full_name,node.location.name,node.location.slug,hashtags,Likes,Date,Datetime,date_obj
57,0,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,"FÚKUDON 一口田\n\nThe right half of ""福"" (fú) (...",Fukudon | Donburi Specialist,Marine Parade Food Centre,marine-parade-food-centre,NaN,43,2/10/2020,2/10/2020 18:49,2020-10-02
56,1,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,Inspired by the quote “Home is where the heart...,Fukudon | Donburi Specialist,Marine Parade Hawker Centre Blk 84,marine-parade-hawker-centre-blk-84,NaN,65,5/10/2020,5/10/2020 19:01,2020-10-05
55,2,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,"In our next few posts, we will be exploring ho...",Fukudon | Donburi Specialist,Marine Parade Central Food Centre,marine-parade-central-food-centre,NaN,49,8/10/2020,8/10/2020 21:04,2020-10-08
54,3,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,This post focuses on two key factors in creati...,Fukudon | Donburi Specialist,Marine Parade Hawker Centre Blk 84,marine-parade-hawker-centre-blk-84,NaN,60,9/10/2020,9/10/2020 18:34,2020-10-09
53,4,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,Renovation from the previous workspace has jus...,Fukudon | Donburi Specialist,Marine Parade Food Centre,marine-parade-food-centre,NaN,55,11/10/2020,11/10/2020 17:55,2020-10-11


# Plot Linear Regression

In [77]:
instagram_df["dates_string"] = instagram_df["date_obj"].apply(lambda x:str(x))

In [78]:
instagram_df.head()

,Unnamed: 0,node.display_url,node.edge_media_to_caption.edges,node.owner.full_name,node.location.name,node.location.slug,hashtags,Likes,Date,Datetime,date_obj,dates_string
57,0,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,"FÚKUDON 一口田\n\nThe right half of ""福"" (fú) (...",Fukudon | Donburi Specialist,Marine Parade Food Centre,marine-parade-food-centre,NaN,43,2/10/2020,2/10/2020 18:49,2020-10-02,2020-10-02
56,1,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,Inspired by the quote “Home is where the heart...,Fukudon | Donburi Specialist,Marine Parade Hawker Centre Blk 84,marine-parade-hawker-centre-blk-84,NaN,65,5/10/2020,5/10/2020 19:01,2020-10-05,2020-10-05
55,2,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,"In our next few posts, we will be exploring ho...",Fukudon | Donburi Specialist,Marine Parade Central Food Centre,marine-parade-central-food-centre,NaN,49,8/10/2020,8/10/2020 21:04,2020-10-08,2020-10-08
54,3,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,This post focuses on two key factors in creati...,Fukudon | Donburi Specialist,Marine Parade Hawker Centre Blk 84,marine-parade-hawker-centre-blk-84,NaN,60,9/10/2020,9/10/2020 18:34,2020-10-09,2020-10-09
53,4,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,Renovation from the previous workspace has jus...,Fukudon | Donburi Specialist,Marine Parade Food Centre,marine-parade-food-centre,NaN,55,11/10/2020,11/10/2020 17:55,2020-10-11,2020-10-11


In [79]:
instagram_df["date_obj"] = pd.to_datetime(instagram_df["date_obj"])

In [80]:
fig = px.scatter(instagram_df, x="date_obj", y="Likes",  trendline="ols")

In [81]:
fig.show()

# Calculate variance

In [82]:
instagram_df.head()

,Unnamed: 0,node.display_url,node.edge_media_to_caption.edges,node.owner.full_name,node.location.name,node.location.slug,hashtags,Likes,Date,Datetime,date_obj,dates_string
57,0,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,"FÚKUDON 一口田\n\nThe right half of ""福"" (fú) (...",Fukudon | Donburi Specialist,Marine Parade Food Centre,marine-parade-food-centre,NaN,43,2/10/2020,2/10/2020 18:49,2020-10-02,2020-10-02
56,1,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,Inspired by the quote “Home is where the heart...,Fukudon | Donburi Specialist,Marine Parade Hawker Centre Blk 84,marine-parade-hawker-centre-blk-84,NaN,65,5/10/2020,5/10/2020 19:01,2020-10-05,2020-10-05
55,2,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,"In our next few posts, we will be exploring ho...",Fukudon | Donburi Specialist,Marine Parade Central Food Centre,marine-parade-central-food-centre,NaN,49,8/10/2020,8/10/2020 21:04,2020-10-08,2020-10-08
54,3,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,This post focuses on two key factors in creati...,Fukudon | Donburi Specialist,Marine Parade Hawker Centre Blk 84,marine-parade-hawker-centre-blk-84,NaN,60,9/10/2020,9/10/2020 18:34,2020-10-09,2020-10-09
53,4,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,Renovation from the previous workspace has jus...,Fukudon | Donburi Specialist,Marine Parade Food Centre,marine-parade-food-centre,NaN,55,11/10/2020,11/10/2020 17:55,2020-10-11,2020-10-11


In [83]:
instagram_df["date_float"] = pd.to_datetime(instagram_df['date_obj'])
instagram_df['date_float'] = instagram_df['date_obj'].map(datetime.toordinal)

In [84]:
import statsmodels.api as sm

x = instagram_df["date_float"].tolist()
y = instagram_df["Likes"].tolist()

x = sm.add_constant(x)
result = sm.OLS(y,x).fit()

In [85]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.153
Model:                            OLS   Adj. R-squared:                  0.138
Method:                 Least Squares   F-statistic:                     10.12
Date:                Mon, 28 Feb 2022   Prob (F-statistic):            0.00239
Time:                        23:11:42   Log-Likelihood:                -276.08
No. Observations:                  58   AIC:                             556.2
Df Residuals:                      56   BIC:                             560.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.754e+04   1.49e+04      3.184      0.0

In [86]:
const = result.params[0]
slope = result.params[1]

def calculate_expected(date_float):
    return slope*(date_float) + const

In [87]:
instagram_df["variance"] = instagram_df["date_float"].apply(calculate_expected)
instagram_df["variance"] = instagram_df["Likes"] - instagram_df["variance"]

In [88]:
instagram_df.head()

,Unnamed: 0,node.display_url,node.edge_media_to_caption.edges,node.owner.full_name,node.location.name,node.location.slug,hashtags,Likes,Date,Datetime,date_obj,dates_string,date_float,variance
57,0,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,"FÚKUDON 一口田\n\nThe right half of ""福"" (fú) (...",Fukudon | Donburi Specialist,Marine Parade Food Centre,marine-parade-food-centre,NaN,43,2/10/2020,2/10/2020 18:49,2020-10-02,2020-10-02,737700,-17.514829
56,1,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,Inspired by the quote “Home is where the heart...,Fukudon | Donburi Specialist,Marine Parade Hawker Centre Blk 84,marine-parade-hawker-centre-blk-84,NaN,65,5/10/2020,5/10/2020 19:01,2020-10-05,2020-10-05,737703,4.678265
55,2,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,"In our next few posts, we will be exploring ho...",Fukudon | Donburi Specialist,Marine Parade Central Food Centre,marine-parade-central-food-centre,NaN,49,8/10/2020,8/10/2020 21:04,2020-10-08,2020-10-08,737706,-11.128641
54,3,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,This post focuses on two key factors in creati...,Fukudon | Donburi Specialist,Marine Parade Hawker Centre Blk 84,marine-parade-hawker-centre-blk-84,NaN,60,9/10/2020,9/10/2020 18:34,2020-10-09,2020-10-09,737707,-0.064277
53,4,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,Renovation from the previous workspace has jus...,Fukudon | Donburi Specialist,Marine Parade Food Centre,marine-parade-food-centre,NaN,55,11/10/2020,11/10/2020 17:55,2020-10-11,2020-10-11,737709,-4.935547


# Feature Engineering for Day and Time
<h3>We will engineer three different features to look at:</h3><br>
<b>1) Day</b> (E.g. Monday, Tuesday) <br>
<b>2) Timing category</b> (E.g. Morning, Evening) -> Definition for category defined below.<br>
<b>3) Compound period </b>(E.g. Monday Morning, Tuesday Evening)

## Timing categorization definition
<b>Midnight / Early Morning:</b> 0000 - 0659 <br>
<b>Morning:</b> 0700 - 1159<br>
<b>Noon:</b> 1200 - 1659<br>
<b>Evening:</b> 1700 - 2059<br>
<b>Night:</b> 2100 - 2359

In [89]:
def get_day_from_date_obj(date_obj):
    return calendar.day_name[date_obj.weekday()]

instagram_df["Day"] = instagram_df["date_obj"].apply(get_day_from_date_obj)

In [90]:
def get_timing_category(datetime_string):
    hour = int(datetime_string.split()[1].split(":")[0])
    if hour >= 0 and hour <= 6:
        return "Midnight / Early Morning"
    if hour >= 7 and hour <= 11:
        return "Morning"
    if hour >= 12 and hour <= 16:
        return "Noon"
    if hour >= 17 and hour <= 20:
        return "Evening"
    if hour >= 21 and hour <= 23:
        return "Night"
instagram_df["time_category"] = instagram_df["Datetime"].apply(get_timing_category)

In [91]:
instagram_df["compound_period"] = instagram_df["Day"] + " " + instagram_df["time_category"]

In [92]:
instagram_df.head()

,Unnamed: 0,node.display_url,node.edge_media_to_caption.edges,node.owner.full_name,node.location.name,node.location.slug,hashtags,Likes,Date,Datetime,date_obj,dates_string,date_float,variance,weekday,time_category,compound_period
57,0,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,"FÚKUDON 一口田\n\nThe right half of ""福"" (fú) (...",Fukudon | Donburi Specialist,Marine Parade Food Centre,marine-parade-food-centre,NaN,43,2/10/2020,2/10/2020 18:49,2020-10-02,2020-10-02,737700,-17.514829,Friday,Evening,Friday Evening
56,1,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,Inspired by the quote “Home is where the heart...,Fukudon | Donburi Specialist,Marine Parade Hawker Centre Blk 84,marine-parade-hawker-centre-blk-84,NaN,65,5/10/2020,5/10/2020 19:01,2020-10-05,2020-10-05,737703,4.678265,Monday,Evening,Monday Evening
55,2,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,"In our next few posts, we will be exploring ho...",Fukudon | Donburi Specialist,Marine Parade Central Food Centre,marine-parade-central-food-centre,NaN,49,8/10/2020,8/10/2020 21:04,2020-10-08,2020-10-08,737706,-11.128641,Thursday,Night,Thursday Night
54,3,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,This post focuses on two key factors in creati...,Fukudon | Donburi Specialist,Marine Parade Hawker Centre Blk 84,marine-parade-hawker-centre-blk-84,NaN,60,9/10/2020,9/10/2020 18:34,2020-10-09,2020-10-09,737707,-0.064277,Friday,Evening,Friday Evening
53,4,https://instagram.fsin5-1.fna.fbcdn.net/v/t51....,Renovation from the previous workspace has jus...,Fukudon | Donburi Specialist,Marine Parade Food Centre,marine-parade-food-centre,NaN,55,11/10/2020,11/10/2020 17:55,2020-10-11,2020-10-11,737709,-4.935547,Sunday,Evening,Sunday Evening


# Aggregate New Features & Create Visualizations

## By Day

In [93]:
aggregate_weekday_df = instagram_df.groupby(["Day"], as_index=False).mean()

In [96]:
aggregate_weekday_df = aggregate_weekday_df.reindex([1,5,6,4,0,2,3])
aggregate_weekday_df

,weekday,Unnamed: 0,Likes,date_float,variance
1,Monday,26.166667,46.666667,737942.166667,1.738823
5,Tuesday,25.875000,34.375000,737922.750000,-11.802591
6,Wednesday,36.200000,33.900000,738045.900000,-4.351078
4,Thursday,27.000000,65.000000,737963.444444,21.441695
0,Friday,26.666667,45.833333,737942.083333,0.900126
2,Saturday,33.500000,39.500000,737977.500000,-3.153624
3,Sunday,27.000000,37.272727,737962.272727,-6.360995


In [97]:
fig = px.bar(aggregate_weekday_df, x='Day', y='variance')
fig.show()

## By Timing Category

In [100]:
aggregate_time_df = instagram_df.groupby(["time_category"], as_index=False).mean()
aggregate_time_df = aggregate_time_df.reindex([1,2,4,0,3])

In [103]:
aggregate_time_df

,time_category,Unnamed: 0,Likes,date_float,variance
1,Midnight / Early Morning,8.000000,61.000000,737717.000000,1.579371
2,Morning,28.166667,42.500000,737949.166667,-1.977290
4,Noon,37.666667,42.740741,738068.555556,5.947881
0,Evening,19.869565,43.434783,737871.347826,-6.051295
3,Night,2.000000,49.000000,737706.000000,-11.128641


In [104]:
fig = px.bar(aggregate_time_df, x='time_category', y='variance')
fig.show()

## By Compound Period

In [105]:
aggregate_period_df = instagram_df.groupby(["compound_period"], as_index=False).mean()

In [107]:
aggregate_period_df = aggregate_period_df.sort_values(by='variance', ascending=False)

In [109]:
fig = px.bar(aggregate_period_df, x='compound_period', y='variance')
fig.show()

# Conclusion

Generally, it looks like post engagement is the highest when posts are made around <b>afternoon</b> and <b>surrounding the weekends</b>. <br>
This could be because of 2 reasons:
<ol>
    <li>Users are most active around that timing</li>
    <li>While users may be active around weekends